In [1]:
import numpy as np
import pandas as pd

In [5]:
!pip install geopy

     |████████████████████████████████| 112kB 89kB/s eta 0:00:011


In [107]:
from geopy.distance import geodesic

print(geodesic((30.72702,104.07513), (30.72672,104.07504)).m) #计算两个坐标直线距离

34.3582693897104


In [2]:
df = pd.read_csv('/Users/zhehaoyu/Desktop/MCM/模考/D/代码/1/gps_20161101') # 导入csv格式文件中的数据

df.columns = ['Driver ID','Order ID','Time','Longitude','Latitude']
df['Time'] = pd.to_datetime(df['Time'],unit='s')

In [290]:
class Order:
    def __init__(self, Id, begin, end):
        self.order_id = Id
        self.begin = begin
        self.end = end
        
        self.vialog = 0
        for i in range(self.begin+1, self.end):
            self.vialog = self.vialog + geodesic((df['Latitude'][i],df['Longitude'][i]),
                                                 (df['Latitude'][i-1],df['Longitude'][i-1])).m
        self.time = (df['Time'][end-1] - df['Time'][begin]).seconds

In [301]:
class Driver:
    def __init__(self, Id, begin, end):
        self.driver_id = Id
        self.begin = begin
        self.end = end
        
        self.orders = []
        
        start = self.begin
        order_id = df['Order ID'][start]
        for i in range(self.begin+1, self.end):
            if order_id != df['Order ID'][i]:
                
                if i != start+1: # the time is not zero
                    #print('order id: '+str(order_id)+' begin: '+str(start)+' end: '+str(i))
                    self.orders.append(Order(order_id, start, i))
                    order_id = df['Order ID'][i]
                    start = i
                    
                else: # ignore this order
                    order_id = df['Order ID'][i]
                    start = i
        if self.end != start+1:           
            #print('order id: '+str(order_id)+' begin: '+str(start)+' end: '+str(self.end))
            self.orders.append(Order(order_id, start, self.end))
        
        self.order_num = len(self.orders)
        self.driving_vialog = 0
        self.speed = -1
        self.driving_time = 0
        for order in self.orders:
            self.driving_time += order.time
            self.driving_vialog += order.vialog 
        
        if self.driving_time != 0:
            self.speed = self.driving_vialog/self.driving_time

In [302]:
def create_driver(df):
    count = 0
    driver_id = df['Driver ID'][0]
    begin = 0
    drivers = []
    for i in range(df.shape[0]):
        if driver_id != df['Driver ID'][i]:
            if i % 50 == 1:
                #print('driver begin: '+str(begin)+' end: '+str(i))
                drivers.append(Driver(df['Driver ID'][begin], begin, i))
            # update
            begin = i
            driver_id = df['Driver ID'][i]
    drivers.append(Driver(df['Driver ID'][begin], begin, df.shape[0]))
    return drivers

In [303]:
drivers = create_driver(df)

In [304]:
len(drivers)

722

In [309]:
speed = 0
tot_time = 0
mileage = 0
order_num = 0

for driver in drivers:
    speed = speed + driver.speed
    tot_time = tot_time + driver.driving_time
    mileage = mileage + driver.driving_vialog
    order_num = order_num + driver.order_num
    
speed = speed/len(drivers)

In [310]:
print('average speed: '+str(speed))
print('total driving time: '+str(tot_time))
print('total mileage: '+str(mileage))
print('number of orders: '+str(order_num))

average speed: 6.48594960802739
total driving time: 3669796
total mileage: 13653297.830969684
number of orders: 4069


In [341]:
alpha = 40
beta = 10
gamma = 15
bcost = 50

In [342]:
rev = alpha*mileage
cost = bcost*order_num + beta*mileage + gamma*tot_time

eff = (rev - cost)/rev
print(rev)
print(eff)

546131913.2387874
0.6488332513433569


,Driver ID,Order ID,Time,Longitude,Latitude
0,8f20c9188561b796ef8e26196de30be4,39a096b71376b82f35732eff6d95779b,2016-11-01 02:59:10,104.07513,30.72702
1,8f20c9188561b796ef8e26196de30be4,39a096b71376b82f35732eff6d95779b,2016-11-01 02:59:14,104.07504,30.72672
2,8f20c9188561b796ef8e26196de30be4,39a096b71376b82f35732eff6d95779b,2016-11-01 02:59:16,104.07497,30.72630
3,8f20c9188561b796ef8e26196de30be4,39a096b71376b82f35732eff6d95779b,2016-11-01 02:59:19,104.07497,30.72582
4,8f20c9188561b796ef8e26196de30be4,39a096b71376b82f35732eff6d95779b,2016-11-01 02:59:22,104.07496,30.72544
5,8f20c9188561b796ef8e26196de30be4,39a096b71376b82f35732eff6d95779b,2016-11-01 02:59:28,104.07489,30.72487
6,8f20c9188561b796ef8e26196de30be4,39a096b71376b82f35732eff6d95779b,2016-11-01 02:59:31,104.07476,30.72456
7,8f20c9188561b796ef8e26196de30be4,39a096b71376b82f35732eff6d95779b,2016-11-01 02:59:34,104.07457,30.72434
8,8f20c9188561b796ef8e26196de30be4,39a096b71376b82f35732eff6d95779b,2016-11-01 02:59:37,104.07434,30.72406
9,8f20c9188561b796ef8e26196de30be4,39a096b71376b82f35732eff6d95779b,2016-11-01 02:59:40,104.07422,30.72379
